In [1]:
from textfile_data import txt_parser
from plot_classes  import *
from matplotlib import transforms
from math import ceil

In [2]:
data = txt_parser('tHbb_v31_v3.txt')
numeric_data = data.astype(float)
data0      = numeric_data[numeric_data[1] == 0]
data1      = numeric_data[(numeric_data[1] == 1) & (numeric_data.index != 'gamma_stat_SR_bin_9')]
data_gamma = numeric_data[numeric_data.index == 'gamma_stat_SR_bin_9']

data0_1 = data0.iloc[0:20,:]
data0_2 = data0.iloc[40:60,:]

In [3]:
"""
FIGSIZE DEPENDENT ON DATA SIZE + MATPLOTLIB + PANDAS
"""

from plot_classes import EmptyPlot


class PythPullPlot(EmptyPlot):

    
    def __init__(self, pd_obj, rcp_kw={}, **kwargs):
        # need to take in object in constructor if we want figure size to change based on data

        super().__init__(**kwargs)
        self.fill(pd_obj)
        
        # user can control size or let it be calculated automatically
        if 'size' in kwargs.keys():
            print("*Manually changing the figure size*") # need logger
        else:
            self.set_figsize()

        self.create_canvas()
        self.make_grid()
        self.ax = self.make_subplot(0, 1, 0, 1)
        self.set_color() # set default colormap
        self.use_custom_range = False

        # set default rcParams for the plot
        self.rcp_dict = {'xaxis.labellocation': 'center', # location of x label w.r.t. x axis [WORKS AUTOMATICALLY]
                         'axes.labelpad'      : 1, # distance of axis label from axis tick labels [DOESNT WORK AUTOMATICALLY]
                         'font.size'          : 8, # x, y label AND ticks label font size [DOESNT WORK AUTOMATICALLY]
                         'axes.titlesize'     : 20, # master title size [DOESNT WORK AUTOMATICALLY]
                        }
        
        # update dict if user has passed any rcParam values in the constructor
        self.rcp_dict.update({k: v for k, v in rcp_kw.items() if (k in mpl.rcParams)})
        
        # set parameters
        self.config_rcParams(self.rcp_dict)
        

    def set_figsize(self):
        
        # variable figure length based on number of data points
        n = self.nvariables
        ly = 0.4012*n**(0.763) # empiric formula from manually measured numbers and fitted with excel
        self.figsize = (2,ly)  # *** though, starts to deviate at around n > 140

        
    def fill(self, pd_obj):
        
        if isinstance(pd_obj, list):
            self.data_container = pd_obj
            if len(pd_obj) > 1:
                self.generate_cells()
        else:
            self.data_container.append(pd_obj)
        
        self.active_data = self.data_container[0]
        self.store_data()
    
    
    def store_data(self, index):
        
        self.values      = self.active_data[1] # pandas series
        self.pos_err     = self.active_data[2] # pandas series
        self.neg_err     =-self.active_data[3] # pandas series
        self.data_labels = self.active_data.index.to_list() # list
        self.nvariables  = len(self.values)
        
        # central value of the pull plot
        self.center = self.values[0]
    
    
    def generate_cells(self):
        
        # print(self.data_container[0].iloc[0].to_list())
        cellsize = len(self.data_container) # how many entries one cell contains
        cells = [[self.data_container[j].iloc[i].to_list() for j in range(cellsize)] for i in range(len(self.data_container[0]))]
        print(cells)
        gap = 0.1
        
        
    
    
    def set_yaxis(self, **yticks_kw):
        
        # set numerical limits of plot
        self.ax.set_ylim(-1, self.nvariables)
        
        # put ticks on right
        self.ax.yaxis.tick_right()
        
        # put variable names onto axis
        self.ax.set_yticks(self.ypos, labels=self.data_labels, fontsize=self.rcp_dict['font.size'], **yticks_kw)
        
    
    def set_xaxis(self, **xtickslabel_kw):
        # xtickslabel_kw from: https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xticklabels.html
        
        if self.use_custom_range: # if user provided x axis ranges
            xmin = self.user_xmin
            xmax = self.user_xmax
            
        else:
            # check the longest line data contains
            longest = ceil(max([max(self.neg_err), max(self.pos_err)]))
            nice_range = longest + 2
            xmin = -nice_range+self.center
            xmax = nice_range+self.center

        self.ax.set_xlim(xmin, xmax)
        self.ax.set_xticks(np.arange(xmin, xmax+1))
        
        self.ax.set_xticklabels([str(int(x)) for x in np.arange(xmin, xmax+1)], fontsize=self.rcp_dict['font.size'],
                                **xtickslabel_kw)
                
        
    def set_rangex(self, range_list):
        # allow user to enter custom range for the x axis
        # range_list = [xmin, xmax]
        
        if isinstance(range_list, list) and len(range_list) == 2:
            self.range_list = range_list
            self.user_xmin = self.range_list[0]
            self.user_xmax = self.range_list[1]
            
            self.use_custom_range = True
    
    
    def axes_labels(self, fontsize, **titles_kw):
        # set axis titles according to user input
        
        self.fill_titlesdict(**titles_kw)
        self.set_xtitles(self.ax, 'xmain', fontsize=fontsize, labelpad=self.rcp_dict['axes.labelpad'])
    
    
    def pull_plot(self, **errorbar_kw):
        # errorbar_kw are keyword argyments from the matplotlib function ax.errorbar:
        # https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.errorbar.html
        
        # define base transformations and rotation transformations
        base = self.ax.transData
        rot = transforms.Affine2D().rotate_deg(90)
        
        # create arrays for errobar plot
        self.ypos = np.arange(self.nvariables)
        self.xerrs = pd.concat([self.neg_err, self.pos_err], axis=1).T
        
        # default errorbar kw arguments
        self.errorbar_kw = {'color'     : 'k',
                            'fmt'       : 'o',
                            'markersize': 3,
                            'elinewidth': 1
                           }
        self.errorbar_kw.update(errorbar_kw)

        # make plot
        self.ax.errorbar(self.values, self.ypos, xerr=self.xerrs, **self.errorbar_kw)
        
        # put yellow and green bands
        self.ax.fill_between([-2+self.center, 2+self.center], -1, self.nvariables, color='yellow')
        self.ax.fill_between([-1+self.center, 1+self.center], -1, self.nvariables, color='limegreen')
        
        # default y axis
        self.set_yaxis()
        
        # default x axis
        self.set_xaxis()
        
        # remove axis ticks for left and right axis of plot
        self.ax.tick_params(which='both', bottom=True, left=False, top=True, right=False)
        
        # set title
        self.ax.set_title(self.master_title, fontsize=self.rcp_dict['axes.titlesize'])
        
        # put atlas logo
        # hep.atlas.text(loc=4, ax=self.ax) # cannot position it properly

        
    def display(self, save_name='', dpi=1000, **errorbar_kw):
        
        self.pull_plot(**errorbar_kw)
        # self.fig.set_tight_layout(True) # tightlayout might be useful?
        
        if save_name:
            self.saveimage(save_name, dpi)

In [4]:
my_dict={'axes.titlesize': 15}

hist1 = PythPullPlot([data0_1, data0_2], title='Master Title', rcp_kw=my_dict)
# hist1.set_rangex([-6, 11])
hist1.axes_labels(12, xmain=r'$(\hat{\theta}-\theta)/\Delta\theta$')
hist1.display()

[[[0.0, 0.976589, -0.976589], [0.0, 0.976658, -0.976658]], [[0.0, 0.976008, -0.976008], [0.0, 0.976323, -0.976323]], [[0.0, 0.976417, -0.976417], [0.0, 0.976448, -0.976448]], [[0.0, 0.975801, -0.975801], [0.0, 0.975923, -0.975923]], [[0.0, 0.81224, -0.81224], [0.0, 0.976811, -0.976811]], [[0.0, 0.374338, -0.374338], [0.0, 0.976616, -0.976616]], [[0.0, 0.605396, -0.605396], [0.0, 0.673321, -0.673321]], [[0.0, 0.87797, -0.87797], [0.0, 0.549165, -0.549165]], [[0.0, 0.976468, -0.976468], [0.0, 0.976706, -0.976706]], [[0.0, 0.976397, -0.976397], [0.0, 0.976399, -0.976399]], [[0.0, 0.976381, -0.976381], [0.0, 0.976194, -0.976194]], [[0.0, 0.826946, -0.826946], [0.0, 0.947438, -0.947438]], [[0.0, 0.972897, -0.972897], [0.0, 0.277021, -0.277021]], [[0.0, 0.976467, -0.976467], [0.0, 0.490465, -0.490465]], [[0.0, 0.97647, -0.97647], [0.0, 0.762304, -0.762304]], [[0.0, 0.961861, -0.961861], [0.0, 0.602441, -0.602441]], [[0.0, 0.976405, -0.976405], [0.0, 0.975999, -0.975999]], [[0.0, 0.219555, -0

TypeError: store_data() missing 1 required positional argument: 'index'